In [1]:
!source ~/.bash_profile

In [2]:
import numpy as np
import pandas as pd
import os
import re
from tensorflow.keras.preprocessing import image
import facenet
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

2023-04-05 08:10:53.425223: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
db_path = '/Users/santhoshnarayanan/Documents/Turing/GPU/MOSIP/data/testset2'
celebs  = []
 
for r, d, f in os.walk(db_path): # r=root, d=directories, f = files
    for file in f:
        if ('.jpg' in file):
            exact_path = r + "/" + file
            celebs.append(exact_path)

In [4]:
target_shape = (160, 160)
imgs = [image.load_img(img, target_size=target_shape) for img in celebs]
X = np.float32([(np.float32(img) - 127.5) / 127.5 for img in imgs])
display(X.shape)

(32000, 160, 160, 3)

In [11]:
checkpoint_path = '/Users/santhoshnarayanan/Documents/Turing/GPU/MOSIP/models/Baseline/vgg'
# create a facenet model
meta_file, ckpt_file = facenet.get_model_filenames(checkpoint_path)
print('Metagraph file: %s' % meta_file)
print('Checkpoint file: %s' % ckpt_file)

Metagraph file: model-20230404-195651.meta
Checkpoint file: model-20230404-195651.ckpt-20


In [12]:
def deducing_metagraph_path(meta_graph_file: str):
    match = re.search(r'^(.*)\.(data-\d*-of-\d*|index|meta)$', meta_graph_file)
    if match is not None:
        deduced_meta_graph_file = match.group(1) + '.meta'
        if not os.path.isfile(deduced_meta_graph_file):
            raise Error('\n\nMetaGraph freezing mechanism was enabled. '
                        '\n{} file does not represent MetaGraph. '
                        '\n{} path to MetaGraph was deduced, but it does not exist'
                        '\n\nModel with MetaGraph consists of 3-4 files:'
                        '\n1. model_name.meta'
                        '\n2. model_name.index'
                        '\n3. model_name.data-00000-of-00001 (digit part may vary)'
                        '\n4. checkpoint (optional)'.format(meta_graph_file, deduced_meta_graph_file))
        else:
            meta_graph_file = deduced_meta_graph_file
    else:
        raise Error('\n\nMetaGraph freezing mechanism was enabled. '
                    '\n{} file does not represent MetaGraph. '
                    '\n\nModel with MetaGraph consists of 3-4 files:'
                    '\n1. model_name.meta'
                    '\n2. model_name.index'
                    '\n3. model_name.data-00000-of-00001 (digit part may vary)'
                    '\n4. checkpoint (optional)'
                    '\n\nTo load this model, simply run:'
                    '\npython3 mo_tf.py --input_meta_graph model_name.meta'
                    ''.format(meta_graph_file))
    return meta_graph_file

def read_file_to_graph_def(graph_def: [tf.GraphDef, tf.MetaGraphDef], graph_file_name: str = "",
                           is_binary: bool = True):
    """
    Reads file to protobuf
    :param graph_def: GraphDef orr MetaGraphDef object to store the network
    :param graph_file_name: path to file with graph
    :param is_binary: flag to switch between binary and test protobuf format of graph file
    :return: GraphDef or MetaGaphDef containing the network with cleared device info.
    """
    try:
        if is_binary:
            with open(graph_file_name, "rb") as f:
                graph_def.ParseFromString(f.read())
        else:
            with open(graph_file_name, "r") as f:
                text_format.Merge(f.read(), graph_def)
        nodes_to_clear_device = graph_def.node if isinstance(graph_def, tf.GraphDef) else graph_def.graph_def.node
        for node in nodes_to_clear_device:
            node.device = ""
    except Exception as e:
        raise FrameworkError(
            'TensorFlow cannot read the model file: "{}" is incorrect TensorFlow model file. '
            '\nThe file should contain one of the following TensorFlow graphs:'
            '\n1. frozen graph in text or binary format'
            '\n2. inference graph for freezing with checkpoint (--input_checkpoint) in text or binary format'
            '\n3. meta graph'
            '\n\nMake sure that --input_model_is_text is provided for a model in text format. '
            'By default, a model is interpreted in binary format. Framework error details: {}. ' +
            refer_to_faq_msg(43),
            graph_file_name,
            str(e)
        ) from e
    return graph_def

In [13]:
with tf.Graph().as_default():
    with tf.Session() as sess:
        meta_graph_file = deducing_metagraph_path(os.path.join(checkpoint_path, meta_file))
        input_meta_graph_def = read_file_to_graph_def(tf.MetaGraphDef(), meta_graph_file, is_binary = True)
        # Since version 2.2 TF can fail with internal error while loading graph from .meta file.
        # It happens because some operation may has an _output_shapes attribute inconsistent with the GraphDef
        # calculated value. To avoid this problem we must delete `_output_shapes` attributes from operations
        for node in input_meta_graph_def.graph_def.node:
            if '_output_shapes' in node.attr:
                del node.attr['_output_shapes']
                    
        saver = tf.train.import_meta_graph(input_meta_graph_def)
        saver.restore(tf.get_default_session(), os.path.join(checkpoint_path, ckpt_file))
        
        images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
        embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
        phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")

        # Run forward pass to calculate embeddings
        feed_dict = {images_placeholder: X[0:80], phase_train_placeholder:False}
        rep = sess.run(embeddings, feed_dict=feed_dict)
        print(rep.shape)
            
        for i in range(1, 400):
            feed_dict = { images_placeholder: X[(80*i):(80*(i+1))], phase_train_placeholder:False}
            repA = sess.run(embeddings, feed_dict=feed_dict)
            rep = np.concatenate((rep, repA), axis=0)
            print(rep.shape)

2023-04-05 08:29:13.077256: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-05 08:29:13.077286: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


INFO:tensorflow:Restoring parameters from /Users/santhoshnarayanan/Documents/Turing/GPU/MOSIP/models2/Full/model-20230404-195651.ckpt-20


2023-04-05 08:29:18.069385: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-05 08:29:19.930018: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


(80, 512)
(160, 512)
(240, 512)
(320, 512)
(400, 512)
(480, 512)
(560, 512)
(640, 512)
(720, 512)
(800, 512)
(880, 512)
(960, 512)
(1040, 512)
(1120, 512)
(1200, 512)
(1280, 512)
(1360, 512)
(1440, 512)
(1520, 512)
(1600, 512)
(1680, 512)
(1760, 512)
(1840, 512)
(1920, 512)
(2000, 512)
(2080, 512)
(2160, 512)
(2240, 512)
(2320, 512)
(2400, 512)
(2480, 512)
(2560, 512)
(2640, 512)
(2720, 512)
(2800, 512)
(2880, 512)
(2960, 512)
(3040, 512)
(3120, 512)
(3200, 512)
(3280, 512)
(3360, 512)
(3440, 512)
(3520, 512)
(3600, 512)
(3680, 512)
(3760, 512)
(3840, 512)
(3920, 512)
(4000, 512)
(4080, 512)
(4160, 512)
(4240, 512)
(4320, 512)
(4400, 512)
(4480, 512)
(4560, 512)
(4640, 512)
(4720, 512)
(4800, 512)
(4880, 512)
(4960, 512)
(5040, 512)
(5120, 512)
(5200, 512)
(5280, 512)
(5360, 512)
(5440, 512)
(5520, 512)
(5600, 512)
(5680, 512)
(5760, 512)
(5840, 512)
(5920, 512)
(6000, 512)
(6080, 512)
(6160, 512)
(6240, 512)
(6320, 512)
(6400, 512)
(6480, 512)
(6560, 512)
(6640, 512)
(6720, 512)
(6800

In [14]:
df = pd.DataFrame(rep) 
df.insert(loc = 0,
          column = 'celeb',
          value = celebs)
display(df)

,celeb,0,1,2,3,4,5,6,7,8,...,502,503,504,505,506,507,508,509,510,511
0,/Users/santhoshnarayanan/Documents/Turing/GPU/...,0.140913,0.020871,-0.013964,-0.005714,0.016393,0.026472,-0.033053,-0.023432,0.002486,...,-0.011166,-0.002652,0.074123,-0.022676,-0.024771,0.026195,0.027847,-0.001614,-0.044211,0.044756
1,/Users/santhoshnarayanan/Documents/Turing/GPU/...,0.037995,0.048820,0.031066,0.027960,0.040582,-0.004255,0.101755,0.071634,0.086596,...,0.074774,-0.083025,0.031071,-0.062660,0.071427,-0.061965,0.020284,0.013158,-0.048352,0.015066
2,/Users/santhoshnarayanan/Documents/Turing/GPU/...,0.028438,0.079346,-0.029471,-0.053542,0.017065,0.059257,-0.085024,0.096279,0.055680,...,0.024013,-0.058146,-0.012174,0.040741,0.035517,0.069107,0.006794,0.043590,0.076894,-0.038523
3,/Users/santhoshnarayanan/Documents/Turing/GPU/...,0.019586,0.014261,0.029243,0.037207,-0.016591,0.060612,0.023974,0.044661,0.005386,...,-0.073187,-0.098848,-0.002306,-0.043729,0.013604,-0.026209,0.075653,-0.025389,-0.042848,-0.051370
4,/Users/santhoshnarayanan/Documents/Turing/GPU/...,0.062072,-0.056812,0.042471,0.002585,0.013636,-0.028059,0.067259,0.002686,0.013643,...,-0.045566,-0.029427,-0.008953,0.006715,0.012756,-0.015098,0.007507,0.001285,-0.014606,-0.032952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31995,/Users/santhoshnarayanan/Documents/Turing/GPU/...,-0.088578,0.046887,-0.062865,0.020126,0.063792,0.000726,0.036415,0.036270,0.049040,...,-0.011748,-0.056202,0.064054,-0.012790,0.017391,0.011818,-0.038471,0.028461,-0.008483,0.013150
31996,/Users/santhoshnarayanan/Documents/Turing/GPU/...,-0.087641,-0.012375,0.023643,-0.026703,-0.050070,-0.027999,-0.033826,0.026767,-0.060402,...,-0.061301,-0.014211,0.053993,0.044482,-0.010010,-0.049437,0.016343,-0.015341,0.005105,-0.016490
31997,/Users/santhoshnarayanan/Documents/Turing/GPU/...,0.010450,-0.040486,-0.095807,-0.031566,-0.096223,0.039675,-0.031280,0.013721,-0.023350,...,-0.019124,0.023780,0.080331,0.006743,-0.051805,-0.008084,0.006430,0.035891,0.027927,0.019425
31998,/Users/santhoshnarayanan/Documents/Turing/GPU/...,-0.010745,-0.027390,0.019613,-0.031781,-0.121209,0.029074,-0.024480,-0.001059,0.017793,...,-0.010119,-0.026567,0.019273,-0.007648,-0.118488,-0.048124,-0.017712,-0.016468,-0.029910,0.062610


In [15]:
df.to_csv('rep_Baseline_FACENET_vgg.csv')